In [60]:
import os
import sys
import urllib.request
import datetime
import time
import json
import pandas as pd

ServiceKey = "RQ6yd8pwrlsAAJVnWM%2FWNLpfYmOgnBdajYK1yLXamt4J2JkmIEe2w13K4M4r%2F2IuiRKZvppl9dENoMw9YXsv7w%3D%3D"

def getRequestUrl(url):
    req = urllib.request.Request(url)
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] url Request Success" %datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] url Request Error : " %(datetime.datetime.now(), url))
        return None

def getTourismStatsItem(yyyymm, nat_cd, ed_cd):
    service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"
    parameters = "?_type=json&serviceKey="+ServiceKey
    parameters += "&YM="+yyyymm
    parameters += "&NAT_CD="+nat_cd
    parameters += "&ED_CD="+ed_cd
    
    url = service_url + parameters
    
    responseDecode = getRequestUrl(url)
    
    if responseDecode == None:
        return None
    else:
        return json.loads(responseDecode)
def getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear):
    jsonResult = []
    result = []
    dataEND = "{0}{1:0>2}"
    isDataEnd = 0
    
    for year in range(nStartYear, nEndYear+1):
        for month in range(1, 13):
            if(isDataEnd == 1): break
            yyyymm = "{0}{1:0>2}".format(str(year), str(month))
            jsonData = getTourismStatsItem(yyyymm, nat_cd, ed_cd)
            
            if(jsonData['response']['header']['resultMsg'] == 'OK'):
                #데이터가 없는 마지막 항목인 경우
                if jsonData['response']['body']['items'] == '':
                    isDataEnd = 1 #데이터 끝 flag 설정
                    dataEND = "{0}{1:0>2}".format(str(year),str(month-1))
                    print("데이터 없음\n제공되는 통계데이터는 %s년 %s월까지 입니다." 
                          %(str(year),str(month-1)))
                    break
                    #jsonData를 출력하여 확인
                print(json.dumps(jsonData, indent = 4, sort_keys = True, ensure_ascii = False))
                natName = jsonData['response']['body']['items']['item']['natKorNm']
                natName = natName.replace(' ','')
                num = jsonData['response']['body']['items']['item']['num']
                ed = jsonData['response']['body']['items']['item']['ed']
                print('[%s_%s : %s]' %(natName, yyyymm, num))
                    
                jsonResult.append({'natName': natName , 'nat_cd': nat_cd, 'yyyymm' : yyyymm , 'visit_cnt': num})
    return(jsonResult, result, natName, ed, dataEND)

def main():
    jsonResult = []
    result = []
    print("<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>")
    nat_cd = input('국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) : ')
    nStartYear =int(input('데이터를 몇 년부터 수집할까요? : '))
    nEndYear = int(input('데이터를 몇 년까지 수집할까요? : '))
    ed_cd = "E"     #E : 방한외래관광객, D : 해외 출국
    
    jsonResult , result ,natName , ed , DataEND = getTourismStatsService(
      nat_cd, ed_cd , nStartYear , nEndYear)
    #파일 저장 json
    if natName == '':
        print("데이터가 전달되지 않았습니다.공공데이터포털의 서비스 상태를 확인하세요")
    else:
        with open('%s_%s_%d_%s.json' %(natName,ed,nStartYear,DataEND), 'w', encoding = 'utf-8') as outfile:
            jsonFile = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii= False)
            outfile.write(jsonFile)
    #csv파일
    columns = ["입국자국가", "국가코드","입국연월", "입국자 수"]
    result_df = pd.DataFrame(result, columns = columns)
    result_df.to_csv('%s_%s_%d_%s.csv' %(natName,ed,nStartYear,DataEND),index = False, encoding = 'cp949')
if __name__ == '__main__':
    main()
    
    

<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>
국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) : 112
데이터를 몇 년부터 수집할까요? : 2022
데이터를 몇 년까지 수집할까요? : 2022
[2022-11-08 12:27:19.176368] url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 112,
                    "natKorNm": "중  국",
                    "num": 9489,
                    "rnum": 1,
                    "ym": 202201
                }
            },
            "numOfRows": 10,
            "pageNo": 1,
            "totalCount": 1
        },
        "header": {
            "resultCode": "0000",
            "resultMsg": "OK"
        }
    }
}
[중국_202201 : 9489]
[2022-11-08 12:27:19.201617] url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 112,
     